In [2]:
from abc import ABC, abstractmethod
from typing import List

class Endereco:
    def __init__(self, rua, numero, cidade, estado, cep):
        self.rua = rua
        self.numero = numero
        self.cidade = cidade
        self.estado = estado
        self.cep = cep

class Cliente:
    def __init__(self, nome, cpf):
        self.nome = nome
        self.cpf = cpf
        self.enderecos = []  
        self.contas = []     

    def adicionar_endereco(self, endereco):
        self.enderecos.append(endereco)

    def adicionarConta(self, conta):
        self.contas.append(conta)

    def removerConta(self, numeroConta):
        self.contas = [conta for conta in self.contas if conta.numero != numeroConta]

    def listarContas(self):
        return self.contas

class ContaBancaria(ABC):
    def __init__(self, titular: Cliente, numero: str):
        self.titular = titular
        self.saldo = 0.0
        self.numero = numero
        self.historico = []

    @abstractmethod
    def debitar(self, valor):
        pass

    def creditar(self, valor):
        self.saldo += valor
        self.historico.append(f"Credito: {valor}")
        return True

    def consultarSaldo(self):
        return self.saldo

    def registrar_historico(self, operacao):
        self.historico.append(operacao)

    def exibir_historico(self):
        return self.historico

class ContaCorrente(ContaBancaria):
    def __init__(self, titular: Cliente, numero: str, limiteChequeEspecial: float):
        super().__init__(titular, numero)
        self.limiteChequeEspecial = limiteChequeEspecial

    def debitar(self, valor):
        if self.saldo + self.limiteChequeEspecial >= valor:
            self.saldo -= valor
            self.registrar_historico(f"Debito: {valor}")
            return True
        else:
            return False

class ContaPoupanca(ContaBancaria):
    def __init__(self, titular: Cliente, numero: str, taxaRendimento: float):
        super().__init__(titular, numero)
        self.taxaRendimento = taxaRendimento
        self.saques_diarios = 0

    def debitar(self, valor):
        if self.saldo >= valor and self.saques_diarios < 3:
            self.saldo -= valor
            self.registrar_historico(f"Debito: {valor}")
            self.saques_diarios += 1
            return True
        else:
            return False

    def aplicarRendimento(self):
        rendimento = self.saldo * self.taxaRendimento
        self.saldo += rendimento
        self.registrar_historico(f"Rendimento: {rendimento}")
        return rendimento

    def reset_saques_diarios(self):
        self.saques_diarios = 0

class Banco:
    def __init__(self):
        self.clientes = []
        self.contas = []

    def criarCliente(self, nome, cpf):
        cliente = Cliente(nome, cpf)
        self.clientes.append(cliente)
        return cliente

    def associarContaCliente(self, cliente: Cliente, conta: ContaBancaria):
        cliente.adicionarConta(conta)
        self.contas.append(conta)

    def deposito(self, conta: ContaBancaria, valor: float):
        conta.creditar(valor)

    def saque(self, conta: ContaBancaria, valor: float):
        return conta.debitar(valor)

    def transferencia(self, contaOrigem: ContaBancaria, contaDestino: ContaBancaria, valor: float):
        if contaOrigem.debitar(valor):
            contaDestino.creditar(valor)
            return True
        else:
            return False

    def encontrarContaPorNumero(self, numero: str):
        for conta in self.contas:
            if conta.numero == numero:
                return conta
        return None

banco = Banco()

cliente1 = banco.criarCliente("Gabriela", "123.456.789-00")
cliente2 = banco.criarCliente("Otávio", "987.654.321-00")

endereco1 = Endereco("Rua A", "123", "São Paulo", "SP", "12345-678")
endereco2 = Endereco("Rua B", "456", "Campinas", "SP", "98765-432")
cliente1.adicionar_endereco(endereco1)
cliente2.adicionar_endereco(endereco2)

conta1 = ContaCorrente(cliente1, "0001", 500.0)
conta2 = ContaPoupanca(cliente1, "0002", 0.05)
conta3 = ContaCorrente(cliente2, "0003", 300.0)

banco.associarContaCliente(cliente1, conta1)
banco.associarContaCliente(cliente1, conta2)
banco.associarContaCliente(cliente2, conta3)

banco.deposito(conta1, 1000.0)
banco.saque(conta1, 200.0)
banco.transferencia(conta1, conta3, 150.0)
conta2.aplicarRendimento()

print(f"Saldo Conta 1: {conta1.consultarSaldo()}")
print(f"Saldo Conta 2: {conta2.consultarSaldo()}")
print(f"Saldo Conta 3: {conta3.consultarSaldo()}")

print(f"Histórico Conta 1: {conta1.exibir_historico()}")
print(f"Histórico Conta 2: {conta2.exibir_historico()}")
print(f"Histórico Conta 3: {conta3.exibir_historico()}")

conta2.reset_saques_diarios()


Saldo Conta 1: 650.0
Saldo Conta 2: 0.0
Saldo Conta 3: 150.0
Histórico Conta 1: ['Credito: 1000.0', 'Debito: 200.0', 'Debito: 150.0']
Histórico Conta 2: ['Rendimento: 0.0']
Histórico Conta 3: ['Credito: 150.0']
